In [1]:
import pandas as pd
import glob, os
pd.set_option('float_format', '{:f}'.format)

dateparse = lambda x: pd.datetime.strptime(x, '%Y%m%d')

pd.options.display.max_columns = 999
pd.options.display.max_rows = 100

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))

import plotly
import plotly.graph_objs as go
plotly.offline.init_notebook_mode(connected=True)

In [2]:
def plot_signal(df, signal_name, stop_legend = False, opacity_cndl = 0.8):
    # рисуем свечной график с объемами
    # signal_name - колонка с сигналом
    # ниже кода для трех состояний сигнала: +1, -1, 0
    # +1 синий, -1 розовый, 0 цвет свечек не меняется

    df['Date_2'] = df['Date'].dt.strftime('%d-%m-%Y')

    color_signal_long = '#0028FF' # цвет сигнала для лонга - синий
    color_signal_short = '#ff00f1' # цвет сигнала для шорта - розовый
    
    mask_long = df[signal_name]==True
#    mask_short = df[signal_name]<0
        
    name_long = 'Long_' + str(round(df[mask_long].shape[0]*100/df.shape[0],1)) + '%'
#    name_short = 'Short_' + str(round(df[mask_short].shape[0]*100/df.shape[0],1)) + '%'
        
    ticker_name = df.Ticker[0]
    data = [ dict(
        name = ticker_name + '_' + signal_name,
        type = 'candlestick',
        x = df['Date_2'],
        yaxis = 'y2',
        open = df['Open'],
        high = df['High'],
        low = df['Low'],
        close = df['Close'],
        opacity = opacity_cndl
    ),
        dict(
        name = name_long,
        type = 'candlestick',
        x = df['Date_2'][mask_long],
        yaxis = 'y2',
        open = df['Open'][mask_long],
        high = df['High'][mask_long],
        low = df['Low'][mask_long],
        close = df['Close'][mask_long],
        hoverinfo='none',
        increasing = dict(line = dict(color = '#0028FF')),
        decreasing = dict(line = dict(color = '#0028FF'))
    )#,
#        dict(
#        name = name_short,
#        type = 'candlestick',
#        x = df['Date_2'][mask_short],
 #       yaxis = 'y2',
 #       open = df['Open'][mask_short],
 #       high = df['High'][mask_short],
  #      low = df['Low'][mask_short],
  #      close = df['Close'][mask_short],
  #      hoverinfo='none',
  #      increasing = dict(line = dict(color = '#ff00f1')),
  #      decreasing = dict(line = dict(color = '#ff00f1'))
#    )
           ]

    layout=dict()

    fig = dict(data=data, layout=layout)

    fig['layout'] = dict()
    #fig['layout']['plot_bgcolor'] = 'rgb(250, 250, 250)'
    #categoryorder = 'category ascending', 
    fig['layout']['xaxis'] = dict(type ='category', 
                                  tickfont=dict(size=10),
                                  rangeselector = dict(visible = True), 
                                  rangeslider = dict(visible = False))
    fig['layout']['yaxis'] = dict(domain = [0, 0.2], 
                                  showticklabels = False, 
                                  autorange = True, 
                                  type = 'log',
#                                  title = 'Volume'
                                 )
    fig['layout']['yaxis2'] = dict(domain = [0.1, 0.95], 
                                   autorange = True, 
                                   type = 'log',
                                   title = ticker_name + ' prices')
    fig['layout']['legend'] = dict(orientation = 'h', 
                                   y = 0.95, x = 0.1, 
                                   yanchor = 'bottom')
    fig['layout']['margin'] = dict(t = 40, b = 40, r = 40, l = 40)
    
    if stop_legend:
        fig['data'].append(dict(x=df['Date_2'], y=df['ATR_sl_long'], type='scatter', yaxis='y2', 
                                line = dict(width = 1), marker=dict(color='#FF4500'), name='SL_long'))
        fig['data'].append(dict(x=df['Date_2'], y=df['ATR_sl_short'], type='scatter', yaxis='y2', 
                            line = dict(width = 1), marker=dict(color='#FF4500'), name='SL_short'))
        fig['data'].append(dict(x=df['Date_2'], y=df['ATR_sp_long'], type='scatter', yaxis='y2', 
                                line = dict(width = 1), marker=dict(color='#008B8B'), name='SP_long'))
        fig['data'].append(dict(x=df['Date_2'], y=df['ATR_sp_short'], type='scatter', yaxis='y2', 
                            line = dict(width = 1), marker=dict(color='#008B8B'), name='SP_short'))


    plotly.offline.iplot(fig)
    return


In [4]:
pd.options.display.max_rows = 1000

path = r'C:\Users\parsh\Desktop\Exchange\data\US_stocks_csv'
file_name = '/NYSE/ABX.csv'
file_ticker = path + file_name

vol_df = pd.read_csv(file_ticker)
vol_df['Date'] = pd.to_datetime(vol_df['Date'], format='%Y-%m-%d', errors='ignore')

# Turtle simple soup

In [5]:
days_for_max = 20 # для расчета колонки с 20дневным максимумом
col_max_name = 'max_'+str(days_for_max)+'_days'

# число дней с текущего, когда сравниваем - пробили 20дневный максимум или нет
# не путать с days_for_max
day_seeking = 20 

# зона "дней назад" (шифта) от текущего, в которую не должен попасть 20дневный максимум
# из колонки col_max_name. На самом деле получается эпти-зона day_empty_zone-1, 
# то есть есть в дне shift(day_empty_zone) 20max может быть
day_empty_zone = 4



In [6]:
# !!!!!!!!!!! Важно!!!!!!!!!!!
# nan присваиваем минус -100000 - потому что не для акций, а индексов могут быть отрицательные значения!!!
# Следовательно когда ищем минимумы - надо присваивать +10000

vol_df[col_max_name] = vol_df['High'][vol_df['High'].rolling(days_for_max).max() == vol_df['High']]
vol_df[col_max_name] = vol_df[col_max_name].fillna(-100000)

In [ ]:
vol_df['max_20max'] = vol_df[col_max_name].shift(4).rolling(17).max()
vol_df['max_high_empty_zone'] = vol_df['High'].shift(1).rolling(3).max()
vol_df['signal_short'] = ((vol_df['High'] > vol_df['max_20max']) & \
                        (vol_df['max_20max'] >= vol_df['max_high_empty_zone']) & \
                        (vol_df['Close'] < vol_df['max_20max']))
vol_df

In [7]:
vol_df['max_20max'] = vol_df[col_max_name].shift(day_empty_zone).rolling(day_seeking-day_empty_zone+1).max()
vol_df['max_high_empty_zone'] = vol_df['High'].shift(1).rolling(day_empty_zone-1).max()
vol_df['signal_short'] = ((vol_df['High'] > vol_df['max_20max']) & \
                        (vol_df['max_20max'] >= vol_df['max_high_empty_zone']) & \
                        (vol_df['Close'] < vol_df['max_20max']))
vol_df


,Ticker,Date,Open,High,Low,Close,Volume,max_20_days,max_20max,max_high_empty_zone,signal_short
0,ABX,1970-01-01 00:00:00.019850201,0.624480,0.624480,0.624480,0.624480,73341,-100000.000000,nan,nan,False
1,ABX,1970-01-01 00:00:00.019850204,0.624480,0.624480,0.624480,0.624480,50762,-100000.000000,nan,nan,False
2,ABX,1970-01-01 00:00:00.019850205,0.624480,0.624480,0.624480,0.624480,18415,-100000.000000,nan,nan,False
3,ABX,1970-01-01 00:00:00.019850206,0.702540,0.702540,0.624480,0.624480,65334,-100000.000000,nan,0.624480,False
4,ABX,1970-01-01 00:00:00.019850207,0.624480,0.624480,0.624480,0.624480,39873,-100000.000000,nan,0.702540,False
5,ABX,1970-01-01 00:00:00.019850208,0.624480,0.624480,0.624480,0.624480,25541,-100000.000000,nan,0.702540,False
6,ABX,1970-01-01 00:00:00.019850211,0.702540,0.702540,0.624480,0.624480,29304,-100000.000000,nan,0.702540,False
7,ABX,1970-01-01 00:00:00.019850212,0.624480,0.624480,0.624480,0.624480,34749,-100000.000000,nan,0.702540,False
8,ABX,1970-01-01 00:00:00.019850213,0.624480,0.624480,0.624480,0.624480,19696,-100000.000000,nan,0.702540,False
9,ABX,1970-01-01 00:00:00.019850214,0.624480,0.702540,0.624480,0.702540,65975,-100000.000000,nan,0.702540,False


In [10]:
vol_df[vol_df['signal_short'] == True].shape

(81, 12)

In [11]:
plot_signal(vol_df, signal_name = 'signal_short')